# Гарри Поттер и предсказание зарплаты методом линейной регрессии

Импортируем всё, что может понадобиться:

In [2]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv и посмотрите на структуру таблицы.

Проведем предобработку текстов: приведем все буквы к нижнему регистру, а все небуквенные и нецифровые символы заменим на пробелы.

Примените TfidfVectorizer для преобразования текстов (колонка FullDescription) в веторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df при создании TfidVectorizer).

In [ ]:

tfidf_transformed_data = # переменная, в которую запишем полученную таблицу после преобразования

Заменим пропуски в столбцах LocationNormalized и ContractTime на специальную строку ’nan’ и закодируем их методом one-hot, а затем объединим две полученные таблицы в общую таблицу признаков. Выберем также и вектор целевой переменной.

In [ ]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)
one_hot_transformed_data = DictVectorizer().fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
X = hstack([tfidf_transformed_data, one_hot_transformed_data])
y = data['SalaryNormalized']

Будем обучать гребневую регрессию (более общий алгоритм линейной регрессии) - используем класс Ridge с параметром alpha=1 и random_state по вариантам. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

Сравните результат с результатами, полученными на деревьях различной глубины (DecisionTreeRegressor). Чему можно верить больше?

# <s>Гарри Поттер</s> Логистическая регрессия и проблема XOR

XOR - это логическая операция, которая возвращает 1, если аргументы не равны, и 0 в обратном случае. Если мы изобразим случайные данные разных классов по принципу XOR, то увидим, что линейно их никак не разделить. 

In [54]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
%matplotlib inline

# сгенерируем синтетические данные, 
variant = # вставьте ваш вариант
rng = np.random.RandomState(variant)
X = rng.randn(300, 2)
y = np.logical_xor(X[:, 0] < 0, X[:, 1] < 0)
plt.scatter(X[:, 0], X[:, 1], c=y);

In [53]:
# Создайте классификатор логистической регрессии и обучите его на тестовой выборке
classifier = 


# Построим график, который покажет, как обучилась логистическая регрессия.
def plot_boundary(clf, X, y):
    xx, yy = np.meshgrid(np.linspace(-3, 3, 50),
                     np.linspace(-3, 3, 50))
    Z = clf.predict_proba(np.vstack((xx.ravel(), yy.ravel())).T)[:, 1].reshape(xx.shape)
    image = plt.imshow(Z, interpolation='nearest',
                           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                           aspect='auto', origin='lower', cmap=plt.cm.PuOr_r)
    contours = plt.contour(xx, yy, Z, levels=[0], linewidths=2,
                               linetypes='--')
    plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired)
    plt.axis([-3, 3, -3, 3])
    plt.colorbar(image)
    
plot_boundary(classifier, X, y)

Видим, что работает так себе. Что если на вход подать не $x_1$ и $x_2$, а некоторые их полиномы?
Преобразуем текущие признаки в 6 других: $1, x_1, x_2, x_1^2, x_1x_2$ и $x_2^2$.

In [52]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
new_classifier = Pipeline([('poly', PolynomialFeatures(degree=2)), ('logit', LogisticRegression())])

# Обучите новый классификатор на тех же данных и посмотрим, как он себя ведет в исходном пространстве признаков.


plot_boundary(new_classifier, X, y)

Стало лучше. Теперь создайте и обучите дерево с максимальной глубиной, равной четырем. Результат оказывается довольно похожим.

In [51]:
from sklearn.tree import DecisionTreeClassifier
tree = 


plot_boundary(tree, X, y)

Сгенерируем данные той же природы для проверки точности наших моделей. У какой модели score получился больше?

In [50]:
X_test = rng.randn(300, 2)
y_test = np.logical_xor(X_test[:, 0] < 0, X_test[:, 1] < 0)
# код